# Importação das Bibliotecas

Aqui estão sendo importadas as bibliotecas necessárias. `pandas` (apelidado de `pd`) é o que vai ajudar na manipulação dos dados, e `files` do `google.colab` permite que a gente faça upload dos arquivos direto no Colab.


In [ ]:
%pip install pandas
import pandas as pd


# Carregando e exibindo a tabela

Aqui, a gente está puxando o arquivo CSV da tabela e guardando ele no DataFrame `df` usando `pd.read_csv`. Depois, ao digitar só `df`, o Colab já imprime a tabela completa pra gente visualizar.


In [ ]:
df = pd.read_csv('TABELA_BIG_DATA_REQ_MAT_202410111115.csv')
df

# Analisando informações da tabela (tipos e valores não nulos)

Nesta célula, utilizamos `df.info()` para obter uma visão geral da tabela. O comando mostra o número de entradas, os nomes das colunas, quantos valores não nulos cada coluna possui, e os tipos de dados (Dtype) de cada coluna. Essas informações ajudam a identificar colunas com dados ausentes e entender o tipo de cada dado, o que é essencial para o processo de limpeza.


In [ ]:
df.info()

# Verificando e removendo linhas duplicadas

Nesta célula, usamos `df.duplicated()` para identificar quaisquer linhas duplicadas na tabela e armazená-las na variável `duplicados`. Em seguida, imprimimos essas linhas para verificar se há duplicatas que precisam ser removidas. No caso desta tabela, a saída indica que não foram encontradas linhas duplicadas (`Empty DataFrame`), então nenhuma ação adicional é necessária.


In [ ]:
duplicados = df[df.duplicated()]

print("Linhas duplicadas:")
print(duplicados)

# Padronizando valores nas colunas numéricas e não numéricas

Aqui, identificamos todas as colunas numéricas da tabela usando `select_dtypes` com os tipos `int64` e `float64`, e armazenamos os nomes dessas colunas em `colunas_numericas`. Para lidar com valores ausentes, substituímos os valores nulos nas colunas numéricas por `-1`, o que facilita a análise posterior. Em seguida, aplicamos `fillna('-1')` a todo o DataFrame para padronizar os valores ausentes nas colunas não numéricas com `'-1'`, criando uma uniformidade nos dados.


In [ ]:
colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns

df[colunas_numericas] = df[colunas_numericas].fillna(-1)
df = df.fillna('-1')

# Exibindo a tabela após padronização dos valores

Exibindo `df` para ver a tabela após a substituição dos valores nulos por `-1`.


In [ ]:
df

# Convertendo coluna para string

Alteramos o tipo da coluna `MOTCANCCODESTR` para string usando `.astype(str)`, o que ajuda a garantir consistência no tipo de dado dessa coluna.


In [ ]:
df['MOTCANCCODESTR'] = df['MOTCANCCODESTR'].astype(str)

# Convertendo colunas de datas para Unix Time
Este código converte as colunas de data especificadas (`DT_CARGA`, `REQMATDATA`, `ITREQMATATENDDATA`) para Unix Time (segundos desde 01/01/1970), se elas existirem no DataFrame `df`.

1. Para cada coluna, o código verifica se ela está no DataFrame.
2. Caso o tipo seja `int64` ou `object`, tenta convertê-la para o formato datetime com `pd.to_datetime`.
3. Se a conversão falhar, exibe um aviso e ignora a coluna.
4. Valores inválidos (NaT) são preenchidos com `-1`.
5. A coluna é convertida para Unix Time e todos os NaTs são substituídos por `-1`.


In [ ]:
colunas_datas = ['DT_CARGA', 'REQMATDATA', 'ITREQMATATENDDATA']

for col in colunas_datas:
    if col in df.columns:
        print(f"Convertendo a coluna '{col}' para Unix time (alterando apenas o dtype)...")

        if df[col].dtype == 'int64' or 'object':
            df[col] = pd.to_datetime(df[col], errors='coerce')

        if not pd.api.types.is_datetime64_any_dtype(df[col]):
            print(f"Atenção: A coluna '{col}' não pôde ser convertida para datetime.")
            continue

        num_invalid = df[col].isna().sum()
        if num_invalid > 0:
            print(f"Atenção: A coluna '{col}' possui {num_invalid} valores inválidos que serão preenchidos com -1.")

        df[col] = df[col].values.astype('int64') // 10**9

        df[col] = df[col].fillna(-1).astype(int)
    else:
        print(f"A coluna '{col}' não existe no DataFrame.")

# Exibindo a tabela após a conversão de datas

Após a conversão das colunas de data para Unix Time e tratamento dos valores ausentes, exibimos `df` para visualizar o DataFrame atualizado e verificar as mudanças feitas.


In [ ]:
df

In [ ]:
df.info()


In [ ]:
pd.set_option('display.max_columns', None)
df.head()

# Salvando o DataFrame em formato Parquet e fazendo download

Primeiro, instalamos a biblioteca `pyarrow`, para caso não esteja instalada, para suportar a exportação do DataFrame em formato Parquet, que é eficiente para armazenamento e leitura de dados. Em seguida, usamos `df.to_parquet()` para salvar o DataFrame limpo como `dados_limpos.parquet`. Para facilitar o download, importamos `files` do `google.colab` e usamos `files.download()` para disponibilizar o arquivo no Colab.


In [ ]:
%pip install pyarrow

df.to_parquet('dados_limpos.parquet', engine='pyarrow')

from google.colab import files
files.download('dados_limpos.parquet')